# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,marsa alam,25.0757,34.8918,26.66,51,0,4.74,EG,1699317196
1,1,albany,42.6001,-73.9662,8.66,63,25,2.24,US,1699317007
2,2,adamstown,-25.0660,-130.1015,22.81,83,100,7.48,PN,1699317196
3,3,mawlamyinegyunn,16.3833,95.2667,26.06,85,100,1.90,MM,1699317197
4,4,hawaiian paradise park,19.5933,-154.9731,27.73,73,40,4.12,US,1699317197


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
# YOUR CODE HERE
import pandas as pd
# Assuming you have a DataFrame named city_data_df with columns 'Lng', 'Lat', 'Humidity', and 'City'

# Define the size of the points based on humidity
#city_data_df['Marker_Size'] = city_data_df['Humidity']

# Create the scatter plot using hvplot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Marker_Size',
    scale=1,
    color='City',
    alpha=0.5,
    frame_width=700,
    frame_height=500
)

#had to remove the tiles and the geo due to having installation issues
# Display the map
map_plot

/Users/christian/anaconda3/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/christian/anaconda3/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/christian/anaconda3/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/christian/anaconda3/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__get

:Scatter   [Lng]   (Lat,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,islamkot,24.6997,70.1770,24.82,37,0,2.21,PK,1699317211
165,165,murzuq,25.9155,13.9184,21.48,22,0,2.98,LY,1699317236
253,253,kidal,18.4411,1.4078,25.27,11,0,1.97,ML,1699317263
276,276,blackwell,36.8045,-97.2828,21.98,71,0,3.60,US,1699317262
297,297,taoudenni,22.6783,-3.9836,26.68,22,0,3.87,ML,1699317276
328,328,al jawf,29.5000,38.7500,21.58,48,0,0.19,SA,1699317284
375,375,altinopolis,-21.0256,-47.3739,22.64,54,0,1.34,BR,1699317294
425,425,tartouss,34.8890,35.8866,22.36,74,0,1.51,SY,1699317308
430,430,al kharijah,25.4514,30.5464,23.07,23,0,4.04,EG,1699317309
448,448,medina,24.4686,39.6142,25.24,44,0,3.09,SA,1699317025


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,islamkot,PK,24.6997,70.1770,37,
165,murzuq,LY,25.9155,13.9184,22,
253,kidal,ML,18.4411,1.4078,11,
276,blackwell,US,36.8045,-97.2828,71,
297,taoudenni,ML,22.6783,-3.9836,22,
328,al jawf,SA,29.5000,38.7500,48,
375,altinopolis,BR,-21.0256,-47.3739,54,
425,tartouss,SY,34.8890,35.8866,74,
430,al kharijah,EG,25.4514,30.5464,23,
448,medina,SA,24.4686,39.6142,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
islamkot - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
kidal - nearest hotel: No hotel found
blackwell - nearest hotel: Holiday Inn Express Blackwell
taoudenni - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
altinopolis - nearest hotel: Vila das Palmeiras
tartouss - nearest hotel: فندق انترادوس
al kharijah - nearest hotel: Kharga Hotel
medina - nearest hotel: The Oberoi Madina
formosa - nearest hotel: Hotel Hostal del Rey
darya khan marri - nearest hotel: No hotel found
tripoli - nearest hotel: الاصيل
al burayqah - nearest hotel: فندق ماس ليبيا


,City,Country,Lat,Lng,Humidity,Hotel Name
53,islamkot,PK,24.6997,70.1770,37,No hotel found
165,murzuq,LY,25.9155,13.9184,22,فندق باريس - مرزق
253,kidal,ML,18.4411,1.4078,11,No hotel found
276,blackwell,US,36.8045,-97.2828,71,Holiday Inn Express Blackwell
297,taoudenni,ML,22.6783,-3.9836,22,No hotel found
328,al jawf,SA,29.5000,38.7500,48,No hotel found
375,altinopolis,BR,-21.0256,-47.3739,54,Vila das Palmeiras
425,tartouss,SY,34.8890,35.8866,74,فندق انترادوس
430,al kharijah,EG,25.4514,30.5464,23,Kharga Hotel
448,medina,SA,24.4686,39.6142,44,The Oberoi Madina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
geoapify_key

'adab2e7a273c4b3bbc268f5e26b68676'

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat",  
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot 

:Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)